# Finetuning BERT for Question Answering



Install HuggingFace Transformers library

In [ ]:
!pip install transformers

## Get Dataset

We will use the Stanford Question Answering Dataset (SQuAD) 2.0. 

Given a **context** and **question**, our model will be expected to correctly *select* the specific part of a text segment as **answer**. It return the token positions (*start* and *end*)of the predicted answer from the context.

We have three key parts in our dataset:
1. Questions — strings containing the question which we will ask our model
2. Contexts — larger segments of text that contain the answers to our questions within them
3. Answers — shorter strings which are ‘extracts’ of the given contexts that provide an answer to our questions


### Download SQuAD

Download *train* and *dev* data json files

In [ ]:
import os
import requests

if not os.path.exists('squad'):
    os.mkdir('squad')

url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'
res = requests.get(f'{url}train-v2.0.json')

#download 'train' and 'dev'
for file in ['train-v2.0.json', 'dev-v2.0.json']:
    # make the request to download data over HTTP
    res = requests.get(f'{url}{file}')
    # write to file
    with open(f'squad/{file}', 'wb') as f:
        for chunk in res.iter_content(chunk_size=4):
            f.write(chunk)

## Prepare Data for BERT 

### Extract relevant data

Extract questions, contexts, and answers from the JSON files into training and validation sets

In [ ]:
import json

def read_squad(path):
    # open JSON file and load intro dictionary
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                # check if we need to be extracting from 'answers' or 'plausible_answers'
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa[access]:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers

# execute our read SQuAD function for training and validation sets
train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

Display one of the context, question and answer

In [ ]:
print('Context: {}'.format(train_contexts[0]))

print('Question: {}'.format(train_questions[0]))

print('Answer: {}'.format(train_answers[0]))

Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question: When did Beyonce start becoming popular?
Answer: {'text': 'in the late 1990s', 'answer_start': 269}


We have 'text' and 'answer_start' in the answer. But to train our model we need to find the start and end of our answer within the context.

So, we do some pre-processing to add an 'answer_end' value to our answers

In [ ]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

         # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
            
# and apply the function to our two answer lists
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
train_answers[:5]

[{'answer_end': 286, 'answer_start': 269, 'text': 'in the late 1990s'},
 {'answer_end': 226, 'answer_start': 207, 'text': 'singing and dancing'},
 {'answer_end': 530, 'answer_start': 526, 'text': '2003'},
 {'answer_end': 180, 'answer_start': 166, 'text': 'Houston, Texas'},
 {'answer_end': 286, 'answer_start': 276, 'text': 'late 1990s'}]

### Encoding

1.  convert our strings into tokens
2.  translate our *answer_start* and *answer_end* indices from character-position to token-position

#### Tokenization

HuggingFace provides in-built tokenizer. We use it to tokenize our **context/question** pairs. 

Tokenizers can accept parallel lists of sequences and encode them together as sequence pairs.





In [ ]:
from transformers import DistilBertTokenizerFast

# initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# tokenize
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

These context-question pairs are now represented as **Encoding** objects. These objects merge each corresponding **context** and **question** strings to create the required Q&A format expected by BERT — which is simply both context and question concatenated but separated with a [SEP] token. 

This concatenated version is stored within the *input_ids* attribute of our **Encoding** object. But, rather than the human-readable text — the data is stored as BERT-readable token IDs.

In [ ]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] beyonce giselle knowles - carter ( / biːˈjɒnseɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best - selling girl groups of all time. their hiatus saw the release of beyonce\'s debut album, dangerously in love ( 2003 ), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number - one singles " crazy in love " and " baby boy ". [SEP] when did beyonce start becoming popular? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

#### Character to Token positions for *Answers*

Next we need to convert our character start/end positions to token start/end positions. When using HuggingFace Fast Tokenizers, we can use the built in char_to_token() method.


In [ ]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

This function adds two more attributes to our **Encoding** objects — *start_positions* and *end_positions*. Each of these is simply a list containing the start/end token positions of the answer that corresponds to their respective question-context pairs.

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

### Convert to PyTorch Dataset

Our data is ready. Let’s just put it in a PyTorch dataset so that we can easily use it for training. 

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

## Fine-tuning 

#### Get the pre-trained model
DistilBert is a small model compared to most transformer models. We can use a DistilBert model with a QA head for training.

In [ ]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

#### Train (fine-tune) the model

The data and model are both ready to go. We can train the model with **Trainer**/**TFTrainer** or we can use native **PyTorch** as shown below. 

In our case, we replace *labels* in the usual Pytorch training with *start_positions* and *end_positions*. 

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

#num_epoch = 20
## TODO: change num_epoch back to 20
num_epoch = 1
train_batch_size = 16
learning_rate = 5e-5

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# move model over to detected device
model.to(device)

# activate training mode of model
model.train()

# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=learning_rate)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

for epoch in range(num_epoch):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)

    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


Epoch 0: 100%|██████████| 8145/8145 [2:01:45<00:00,  1.11it/s, loss=0.95]


#### Save the fine-tuned model

Once done training, we will save our newly fine-tuned model (and tokenizer if needed) to a new directory using the *save_pretrained* method

In [ ]:
model_path = 'finetuned-distilbert-qa'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('finetuned-distilbert-qa/tokenizer_config.json',
 'finetuned-distilbert-qa/special_tokens_map.json',
 'finetuned-distilbert-qa/vocab.txt',
 'finetuned-distilbert-qa/added_tokens.json',
 'finetuned-distilbert-qa/tokenizer.json')

#### Reload the fine-tuned model

When loading our model again, we can use the same *from_pretrained* method from HuggingFace and give it the model_path (path where we saved the model in previous step)

In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

model_path = 'finetuned-distilbert-qa'

model = DistilBertForQuestionAnswering.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)



## Measure Performance

### Evaluation Metric: Accuracy

The simplest metric we can measure here is **accuracy** — more specifically known as ***exact match*** (EM) — did the model get the right start or end token, or not?

### Evaluate Validation Set

We set up a DataLoader — and then loop through each batch. This time though, we call *model.eval()* to switch the behavior of relevant layers from training to inference mode — and use torch.no_grad() to stop PyTorch from calculating model gradients (only needed during training)

In [ ]:
model.to(device)
# switch model out of training mode
model.eval()
# initialize validation set data loader
val_loader = DataLoader(val_dataset, batch_size=16)
# initialize list to store accuracies
acc = []
# loop through batches
for batch in val_loader:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # we will use true positions for accuracy calc
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull prediction tensors out and argmax to get predicted tokens
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
# calculate average accuracy in total
acc = sum(acc)/len(acc)

In [ ]:
print(acc)

0.6385480182926829
